# Psi4 Notebook for educational purposes


This notebook is used to compare the results of Psi4 with the results with my own implementation of the Hartree-Fock method. It helps me to check if my implementation is correct. 

Connect to pc with `Psi4` installation and host jupyter server.

`ssh -L 8080:localhost:8080 username@pcname`

`jupyter notebook --no-browser --port=8080`

In [148]:
# cd /loctmp/dam63759/psi4_work

In [149]:
ls

h2o_test.out*          Notes.md        Psi4_compare.ipynb*  water_HF.xyz*
h2o_testV2.out*        output.dat      psi4_setup_h2.py*    water_HF.xyz.bak*
ipython_h2o_setup.py*  psi.2629.clean  water_HF_std.xyz*


My setup, that is run:

In [150]:
cat psi4_setup_h2.py

import psi4
import numpy as np 

np.set_printoptions(precision=5, linewidth=100, suppress=True)
psi4.set_memory("4 GB")
psi4.core.set_output_file('output.dat', False)
numpy_memory = 4

mol = psi4.geometry("""
0 1
H 0.0 0.0 0.0
H 0.0 0.0 1.4
units bohr
""")

psi4.set_options({"save_jk": "true", "guess": "CORE", "basis": "sto-3g", "DIIS": "false", "SCF_INITIAL_ACCELERATOR": "None", "CFOUR_SCF_MAXCYC": "500", "e_convergence": 1e-8})

wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())

S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

no_basis_funcs: int = S.shape[0]



In [151]:
%run psi4_setup_h2.py

### This is with STO-3G basis set 

In [152]:
V

array([[-1.88044, -1.19483],
       [-1.19483, -1.88044]])

In [153]:
S

array([[1.     , 0.65932],
       [0.65932, 1.     ]])

### This is with 6-311G basis set 

In [154]:
psi4.set_options({"save_jk": "true", "guess": "CORE", "basis": "6-311G", "DIIS": "false", "SCF_INITIAL_ACCELERATOR": "None", "MAXITER": 250, "CFOUR_SCF_MAXCYC": "500", "e_convergence": 1e-8, "CFOUR_CONTRACTION": "SEGMENTED", "SCF__PRINT_BASIS": "true"})

In [155]:
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())
S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V_ne = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

In [156]:
S

array([[1.     , 0.70638, 0.37366, 0.26948, 0.42526, 0.31026],
       [0.70638, 1.     , 0.78896, 0.42526, 0.72664, 0.67697],
       [0.37366, 0.78896, 1.     , 0.31026, 0.67697, 0.90422],
       [0.26948, 0.42526, 0.31026, 1.     , 0.70638, 0.37366],
       [0.42526, 0.72664, 0.67697, 0.70638, 1.     , 0.78896],
       [0.31026, 0.67697, 0.90422, 0.37366, 0.78896, 1.     ]])

In [157]:
T

array([[2.30435, 0.54912, 0.10635, 0.06934, 0.21812, 0.07735],
       [0.54912, 0.48876, 0.18488, 0.21812, 0.27955, 0.14245],
       [0.10635, 0.18488, 0.15411, 0.07735, 0.14245, 0.13   ],
       [0.06934, 0.21812, 0.07735, 2.30435, 0.54912, 0.10635],
       [0.21812, 0.27955, 0.14245, 0.54912, 0.48876, 0.18488],
       [0.07735, 0.14245, 0.13   , 0.10635, 0.18488, 0.15411]])

In [158]:
V_ne

array([[-2.82405, -1.55802, -0.77545, -0.7038 , -0.97076, -0.65609],
       [-1.55802, -1.54664, -1.03651, -0.97076, -1.19542, -0.92183],
       [-0.77545, -1.03651, -0.96188, -0.65609, -0.92183, -0.89488],
       [-0.7038 , -0.97076, -0.65609, -2.82405, -1.55802, -0.77545],
       [-0.97076, -1.19542, -0.92183, -1.55802, -1.54664, -1.03651],
       [-0.65609, -0.92183, -0.89488, -0.77545, -1.03651, -0.96188]])

In [159]:
ERI

array([[[[1.39303, 0.82297, 0.41126, 0.29007, 0.48009, 0.34014],
         [0.82297, 0.8229 , 0.54368, 0.33807, 0.54967, 0.46004],
         [0.41126, 0.54368, 0.49402, 0.22002, 0.40827, 0.4331 ],
         [0.29007, 0.33807, 0.22002, 0.6992 , 0.48072, 0.25081],
         [0.48009, 0.54967, 0.40827, 0.48072, 0.60763, 0.43572],
         [0.34014, 0.46004, 0.4331 , 0.25081, 0.43572, 0.43853]],

        [[0.82297, 0.5097 , 0.25776, 0.18349, 0.29937, 0.21335],
         [0.5097 , 0.54303, 0.36577, 0.22615, 0.3665 , 0.3099 ],
         [0.25776, 0.36577, 0.34012, 0.14922, 0.27791, 0.29863],
         [0.18349, 0.22615, 0.14922, 0.48072, 0.3286 , 0.17126],
         [0.29937, 0.3665 , 0.27791, 0.3286 , 0.41477, 0.29867],
         [0.21335, 0.3099 , 0.29863, 0.17126, 0.29867, 0.30363]],

        [[0.41126, 0.25776, 0.13079, 0.0933 , 0.15168, 0.10829],
         [0.25776, 0.27987, 0.1898 , 0.11698, 0.18956, 0.16089],
         [0.13079, 0.1898 , 0.17804, 0.07757, 0.14483, 0.15641],
         [0.0933 , 0.

In [160]:
S

array([[1.     , 0.70638, 0.37366, 0.26948, 0.42526, 0.31026],
       [0.70638, 1.     , 0.78896, 0.42526, 0.72664, 0.67697],
       [0.37366, 0.78896, 1.     , 0.31026, 0.67697, 0.90422],
       [0.26948, 0.42526, 0.31026, 1.     , 0.70638, 0.37366],
       [0.42526, 0.72664, 0.67697, 0.70638, 1.     , 0.78896],
       [0.31026, 0.67697, 0.90422, 0.37366, 0.78896, 1.     ]])

In [161]:
from scipy.linalg import sqrtm
S_sqrt = sqrtm(S)
S_sqrt_inv = np.linalg.inv(S_sqrt)

In [162]:
S_sqrt_inv

array([[ 1.37587, -0.74986,  0.21741, -0.02169, -0.00035, -0.03645],
       [-0.74986,  2.40629, -1.16249, -0.00035, -0.66585,  0.40979],
       [ 0.21741, -1.16249,  2.83156, -0.03645,  0.40979, -1.672  ],
       [-0.02169, -0.00035, -0.03645,  1.37587, -0.74986,  0.21741],
       [-0.00035, -0.66585,  0.40979, -0.74986,  2.40629, -1.16249],
       [-0.03645,  0.40979, -1.672  ,  0.21741, -1.16249,  2.83156]])

In [163]:
energy = psi4.energy("SCF", molecule=mol)

In [164]:
energy

-1.1279785387000567

### Writing a function to compare the results -> help identify the problem

In [165]:
scf_mrjd_rs_T = np.array(
[[2.304346, 0.549119, 0.106346, -3.160976, 0.322166, 0.088218],
 [0.549119, 0.488760, 0.184881, -9.113568, 0.279547, 0.157027],
 [0.106346, 0.184881, 0.154112, -6.820726, -0.004220, 0.129996],
 [-3.160976, 0.322166, 0.088218, 2.304346, 0.549119, 0.106346],
 [-9.113568, 0.279547, 0.157027, 0.549119, 0.488760, 0.184881],
 [-6.820726, -0.004220, 0.129996, 0.106346, 0.184881, 0.154112]]
)

In [166]:
is_close_T = np.isclose(scf_mrjd_rs_T, T, atol=0.0005)

In [167]:
for i in range(len(is_close)):
    for j in range(len(is_close[i])):
        if is_close[i][j] == False:
            print("T[{}][{}] is not close".format(i,j))

T[0][3] is not close
T[0][4] is not close
T[0][5] is not close
T[1][3] is not close
T[1][5] is not close
T[2][3] is not close
T[2][4] is not close
T[3][0] is not close
T[3][1] is not close
T[3][2] is not close
T[4][0] is not close
T[4][2] is not close
T[5][0] is not close
T[5][1] is not close


In [168]:
scf_mrjd_rs_S = np.array(
[[0.999999, 0.706377, 0.373664, 0.269482, 0.425257, 0.310263],
 [0.706377, 1.000000, 0.788963, 0.425257, 0.726641, 0.676966],
 [0.373664, 0.788963, 1.000000, 0.310263, 0.676966, 0.904217],
 [0.269482, 0.425257, 0.310263, 0.999999, 0.706377, 0.373664],
 [0.425257, 0.726641, 0.676966, 0.706377, 1.000000, 0.788963],
 [0.310263, 0.676966, 0.904217, 0.373664, 0.788963, 1.000000]]
)

In [172]:
all_close_S = np.allclose(scf_mrjd_rs_S, S, atol=0.0005)
all_close_S

True